In [47]:
from pymongo import MongoClient
import requests
from lxml import html,etree
from bs4 import BeautifulSoup, NavigableString, Tag

In [2]:
def get_database():
    CONNECTION_STRING = "mongodb://localhost:27018/local"
    client = MongoClient(CONNECTION_STRING)
    return client['news']
if __name__ == "__main__":   
    dbname = get_database()

In [3]:
lenta_news_collection = dbname["lenta_news"]
mail_ru_news_collection = dbname["mailru_news"]

In [4]:
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}
list_news=[]
for i in range(10):
    req=requests.get("https://lenta.ru/parts/news/"+str(i)+"/", headers=headers)
    list_news.append(html.fromstring(req.text))
   
  

NameError: name 'BeautifulSoup' is not defined

In [5]:
lenta_news=[]
for  i, one_page in enumerate(list_news):
    for j, item in enumerate(one_page.xpath('//body//ul[1]/li[@class="parts-page__item"]')):
            href=item.xpath('a[@rel="noreferrer"]/@href')[0]  if item.xpath('a[@rel="noreferrer"]/@href') else "https://lenta.ru"+item.xpath('a/@href')[0] 
            title=item.xpath('a/h3[@class="card-full-news__title"]/text()')[0]
            time=item.xpath('a/div[@class="card-full-news__info"]/time/text()')[0]
            rubric=(item.xpath('a/div[@class="card-full-news__info"]/svg/use/@*')+item.xpath('a/div[@class="card-full-news__info"]/span/text()'))[0].replace('#ui-label_','')
            myTuple=( {"_id":href,
                       "href":href,
                       "title":title,
                       "rubric":rubric})
            lenta_news.append(myTuple)

In [6]:
from pymongo import UpdateOne
def insert_or_update_news_in_all_rubrics(collection_name, list_name):
    upserts=[ UpdateOne({'_id':x['_id']}, {'$setOnInsert':x}, upsert=True) for x in list_name]
    collection_name.bulk_write(upserts)


In [8]:
insert_or_update_news_in_all_rubrics(lenta_news_collection, lenta_news)
#collection_name.drop()insert_or_update_news_in_all_rubrics

In [10]:
def select_news_by_rubric(collection_name,rubric):
    for item in  collection_name.find({ 'rubric': rubric}):
        print(item)

In [11]:
select_news_by_rubric(collection_name=lenta_news_collection, rubric='Мир')

{'_id': 'https://lenta.ru/news/2023/01/05/minno/', 'href': 'https://lenta.ru/news/2023/01/05/minno/', 'rubric': 'Мир', 'title': 'Минобороны сообщило о перехвате снарядов HIMARS'}
{'_id': 'https://lenta.ru/news/2023/01/05/pu_er/', 'href': 'https://lenta.ru/news/2023/01/05/pu_er/', 'rubric': 'Мир', 'title': 'Путин и Эрдоган обсудили ситуацию на Украине и в Сирии'}
{'_id': 'https://lenta.ru/news/2023/01/05/filippo/', 'href': 'https://lenta.ru/news/2023/01/05/filippo/', 'rubric': 'Мир', 'title': 'Французский политик назвал безумием передачу бронемашин AMX-10RC Украине'}
{'_id': 'https://lenta.ru/news/2023/01/05/russia_nato/', 'href': 'https://lenta.ru/news/2023/01/05/russia_nato/', 'rubric': 'Мир', 'title': 'Экс-полковник британской армии допустил начало войны России и НАТО в 2023 году'}
{'_id': 'https://lenta.ru/news/2023/01/05/nastupl/', 'href': 'https://lenta.ru/news/2023/01/05/nastupl/', 'rubric': 'Мир', 'title': 'Украине предрекли масштабное наступление зимой'}
{'_id': 'https://lenta.

In [63]:
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}
list_mail_news=[]
news_rubric=["https://news.mail.ru/society/","https://news.mail.ru/politics/","https://news.mail.ru/economics/","https://news.mail.ru/incident/","https://news.mail.ru/landing/russian_culture/","https://news.mail.ru/social-media/","https://news.mail.ru/inregions/moscow/90/","https://pogoda.mail.ru/prognoz/moskva/"]
for rubric in news_rubric:
    req=requests.get(rubric, headers=headers)
    list_mail_news.append(BeautifulSoup(req.text, "lxml"))



In [70]:
mail_news=[]
for  i, one_page in enumerate(list_mail_news):
    for j, item in enumerate(one_page.find_all('a', {'class':'newsitem__title link-holder'})):
        if isinstance(item, Tag):
            try:
                #print(item)
                myTuple=( {"_id":item['href'],
                       "href":item['href'],
                       "title":item.span.text,
                       "rubric":(item['href']).split('/')[3]})
                mail_news.append(myTuple)
            except AttributeError:
                continue 
            

In [71]:
insert_or_update_news_in_all_rubrics(mail_ru_news_collection, mail_news)

In [72]:
select_news_by_rubric(collection_name=mail_ru_news_collection, rubric='society')

{'_id': 'https://news.mail.ru/society/54548033/?utm_source=block_popular', 'href': 'https://news.mail.ru/society/54548033/?utm_source=block_popular', 'rubric': 'society', 'title': 'В\xa0Аргентине начался бум родильного туризма среди россиянок'}
{'_id': 'https://news.mail.ru/society/54549056/', 'href': 'https://news.mail.ru/society/54549056/', 'rubric': 'society', 'title': 'В Москве и Подмосковье объявили оранжевый уровень погодной опасности из-за холодов'}
{'_id': 'https://news.mail.ru/society/54547888/', 'href': 'https://news.mail.ru/society/54547888/', 'rubric': 'society', 'title': 'СМИ: принц Гарри заявил о драке со старшим братом Уильямом'}
{'_id': 'https://news.mail.ru/society/54547348/', 'href': 'https://news.mail.ru/society/54547348/', 'rubric': 'society', 'title': 'В некоторых регионах России прогнозируют морозы от 15 до 53 градусов'}
{'_id': 'https://news.mail.ru/society/54544059/', 'href': 'https://news.mail.ru/society/54544059/', 'rubric': 'society', 'title': 'Британский мон